In [35]:
import requests as r
from bs4 import BeautifulSoup as bs
import json 
from queue import Queue
import threading
import re
import time
import random
import os

In [7]:
pageUrl_pattern = '(http(s)?://)(www\.ipeen.com.tw/search/taiwan/000/1-0-0-0/\?p=)(\d+)'
def all_restaurant_list(page_url):
    print(page_url)
    if re.match(pageUrl_pattern, page_url) == None:        
        print('pageURL is not found')
    else:        
        res = r.get(page_url)
        res.encoding="utf-8"
        soup = bs(res.text, 'lxml')

        #因為要同時寫入4個檔案，沒辦法在單一區塊中完成讀寫，所以不使用with open的原因顯而易見。
        #當然，更不能在迴圈裡使用with open，Connection是重量級資源，反覆開關會大幅度的拖慢速度。
        f1 = open('all_restaurant_list_block1.txt', 'a+',encoding='utf8')
        f2 = open('all_restaurant_list_block2.txt', 'a+',encoding='utf8')
        f3 = open('all_restaurant_list_block3.txt', 'a+',encoding='utf8')
        f4 = open('all_restaurant_list_block4.txt', 'a+',encoding='utf8')

        #為了能夠平均每個block的餐廳數量，用來分割檔案的計數使用random來初始化
        #不直接在迴圈裡使用random是因為雖然這樣能更均衡的分配資料，但Python的偽亂數產生器使用Mersenne Twister演算法
        #產生出來的亂數雖然品質很好，但相對也使用較多的資源。考慮到愛評網的餐廳數量，應該用更節省效能的方式來達成類似的工作。
        initialization_count_number = random.randint(1,4)
        count_number = initialization_count_number

        #過濾掉已搬遷、已歇業的店家，取得單一餐廳的餐廳頁面url
        #這種店家都有<span>標籤
        #1.<span class="status">【已搬遷】</span> 2.<span class="status">【已歇業】</span>
        HOST = 'http://www.ipeen.com.tw'
        all_restaurant_in_h3_list = soup.findAll('h3', {'id':re.compile('shop_h3_\d\d?')})
        try: 
            for restaurant in all_restaurant_in_h3_list:
                if not restaurant.span:
                    if count_number%4==1:
                        f1.write(HOST + restaurant.a['href']+'\n')
                    elif count_number%4==2:
                        f2.write(HOST + restaurant.a['href']+'\n')
                    elif count_number%4==3:        
                        f3.write(HOST + restaurant.a['href']+'\n')
                    else:     
                        f4.write(HOST + restaurant.a['href']+'\n')
        #except:
        #    print('[ERROR]IOexception!')
        finally:
            f1.close()
            f2.close()
            f3.close()
            f4.close()

In [10]:
class AWSTimeLimitError(Exception):
    def __init__(self,msg):
        self.message=msg
   
    def __str__(self):
        return self.message

def getExecutionTime(startTime):
    if (time.time() - startTime < 600):
        pass
    else:
        raise AWSTimeLimitError('Time is running out')

In [39]:
def reduce_AllPagesListSplitBlock_function(q,startTime):
    rf = open('all_pages_list_block1.txt', 'r',encoding='utf8')
    pages_str = rf.read()
    if (pages_str==''):
        print('no url!')
        rf.close()
        with open('success_all_pages_list_block1.txt', 'w', encoding='utf8') as wsf:
            wsf.write('success')
        os.remove('all_pages_list_block1.txt')
    else:
        pages_list = pages_str.split('\n')
        for pages in range(len(pages_list)):
            q.put(pages_list.pop())        

In [45]:
if __name__ == '__main__':   
    
    startTime = time.time()  
    
    q = Queue()
    t1 = threading.Thread(target=reduce_AllPagesListSplitBlock_function, args=(q,startTime,)) 
    
    t1.start()  #啟動 t1 線程

    t1.join()  #在 t1線程結束前阻止程式繼續運行

    #確認Queue是否為空，如果不是就用 q.get() 取出值
    
    while not q.empty():
        try:
            getExecutionTime(startTime)
            all_restaurant_list(q.get())
        except AWSTimeLimitError:
            with open('all_pages_list_block1.txt', 'w', encoding='utf8') as wf:
                while not q.empty():
                    page = q.get()
                    wf.write(page + '\n')
    endTime = time.time() 
    totalExecutionTime = str(endTime-startTime)
    print('good')
    print('[INFO]TotalExecutionTime = ' + totalExecutionTime)

no url!
good
[INFO]TotalExecutionTime = 0.004000186920166016
